In [ ]:
import xarray as xr
import geopandas as gpd
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from datetime import datetime, timedelta

In [15]:
datestart, dateend = "2012-01-01", "2012-01-25"
periods = [
#    ("2012-01-01", "2012-01-25"), ("2012-01-26", "2012-02-19"), ("2012-02-20", "2012-03-15"), ("2012-03-16", "2012-04-09"),
#    ("2012-04-10", "2012-05-04"),
#    ("2012-05-05", "2012-05-29"), ("2012-05-30", "2012-06-28"), ("2012-10-27", "2012-11-20"), ("2012-11-21", "2012-12-15"),   
#    ("2012-06-29", "2012-07-28"), ("2012-07-29", "2012-08-27"), ("2012-08-28", "2012-09-26"), ("2012-09-27", "2012-10-26"),
]

In [27]:
experiment_name = 'INALT60.L120-KRS0020'
data_resolution = '1d'

sigma = 15
wx = int(100*6) # rolling window size in x-direction
wy = int(100*6) # rolling window size in y-direction

# detection parameters
params = 'OW0.3_Npix-720-18000' # 'OW0.5_Npix-1296-32400'
Npix_min = 720 #20*6*5
Npix_max = 500*6*6
OW_thr_factor =-0.3

In [28]:
params = f'OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}'

In [29]:
# defined in function below: outpath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-{depth}/'
file_pattern = "Eddies_{time}_{params}_rolling-{wx}.pickle"

In [30]:
if experiment_name.startswith("INALT60"):
    prefix = "2_"
elif experiment_name.startswith("INALT20"):
    prefix = "1_"

In [ ]:
#depth_information = (0,0)

mesh_mask = xr.open_dataset(f'/gxfs_work/geomar/smomw523/smoothed_data/{experiment_name}/{prefix}{experiment_name}_mesh_mask.nc') 
indices = np.concatenate((range(0, 11, 10),range(18, 25, 6),range(29, 34, 4),range(36, 40, 3),range(41, 120, 2)))
depth_information = [(round(mesh_mask.nav_lev.values[i]), i) for i in indices]
print(len(depth_information),depth_information)

IndexError: index 10 is out of bounds for axis 0 with size 1

In [33]:
lon = mesh_mask.nav_lon.data
lat = mesh_mask.nav_lat.data

In [ ]:
def compute_OW_mask(start, end, depth):
    outpath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-{depth}/'
    output_file = outpath + f'Parcels-mask_{start.replace("-", "")}_{end.replace("-", "")}_{params}_rolling-{wx}.nc'
    if os.path.exists(output_file):
        print(f'> {depth}m: {start.replace("-", "")} to {end.replace("-", "")} with {params} already exists. Skip ...')
        return

    time_str_list = []
    mask_list = []
    area_list = []
    type_list = []
    scale_list = []
    
    time_array = pd.date_range(start=start, end=end, freq=data_resolution).to_pydatetime() + timedelta(hours=12)
    for time in time_array:
        time_str = time.strftime("%Y-%m-%d") #T%H
        time_str_list.append(time_str)
        file_path = outpath + file_pattern.format(time=time_str, params=params, depth=depth, wx=wx)
        
        # empty fields
        mask = np.zeros_like(lon, dtype=int)
        area_field = np.zeros_like(lon, dtype=float)
        type_field = np.zeros_like(lon, dtype=int)
        scale_field = np.zeros_like(lon, dtype=float)

        if os.path.exists(file_path):
            with open(file_path, 'rb') as f:
                OW_1 = pickle.load(f)
        
            # all eddies in OW_1
            for i in range(len(OW_1)):
                indices_i = OW_1[i]['eddy_i']
                indices_j = OW_1[i]['eddy_j']
            
                mask[indices_j, indices_i] = 1
                area_field[indices_j, indices_i] = OW_1[i]['area']
                scale_field[indices_j, indices_i] = OW_1[i]['scale']
            
                #type_field[indices_j, indices_i] = OW_1[i]['type']
                if OW_1[i]['type'] == 'anticyclonic':
                    type_field[indices_j, indices_i] = -1
                elif OW_1[i]['type'] == 'cyclonic':
                    type_field[indices_j, indices_i] = 1
        else:
            print(f"Warning: {file_path} not found. Set all fields to 0.")
            
        mask_list.append(mask)
        area_list.append(area_field)
        type_list.append(type_field)
        scale_list.append(scale_field)
        
    # Combine
    OW = xr.Dataset(
        {
            "eddymask": (["time_counter", "y", "x"], np.array(mask_list, dtype=np.int8)),
            "area": (["time_counter", "y", "x"], np.array(area_list, dtype=np.float32)),
            "type": (["time_counter", "y", "x"], np.array(type_list, dtype=np.int8)),
            "scale": (["time_counter", "y", "x"], np.array(scale_list, dtype=np.float32)),
        },
        coords={
            "time_counter": time_array,
            "nav_lon": (["x"], lon[0]),
            "nav_lat": (["y"], lat[:,0]),
        },
    ).chunk({"time_counter": 1, "y": 50, "x": 50})
    
    # Save
    OW.to_netcdf(outpath + f'Parcels-mask_{time_str_list[0].replace("-", "")}_{time_str_list[-1].replace("-", "")}_{params}_rolling-{wx}.nc')

In [35]:
#for datestart, dateend in periods:
for (depth, depth_index) in tqdm(depth_information, desc='depth levels'):
    #print(f"> Processing period: {datestart} to {dateend}, {depth}m ...")
    compute_OW_mask(datestart, dateend, depth)

depth levels: 100%|██████████| 2/2 [01:13<00:00, 36.79s/it]

> 0m: 20120101 to 20120125 with OW0.3_Npix-720-18000 already exists. Skip ...


In [36]:
combined_outpath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/'
combined_filename = f'Parcels-mask_depths-{len(depth_information)}_{datestart.replace("-", "")}_{dateend.replace("-", "")}_{params}_rolling-{wx}.nc'

datasets = []

#for datestart, dateend in periods:
for (depth, depth_index) in tqdm(depth_information, desc='depth levels'):
    depth_outpath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-{depth}/'
    depth_filename = f'Parcels-mask_{datestart.replace("-", "")}_{dateend.replace("-", "")}_{params}_rolling-{wx}.nc'
    depth_filepath = os.path.join(depth_outpath, depth_filename)
    
    if os.path.exists(depth_filepath):
        ds = xr.open_dataset(depth_filepath)
        ds = ds.assign_coords(nav_lev=mesh_mask.nav_lev.values[depth_index]).expand_dims("z")
        datasets.append(ds)
    else:
        print(f"{depth_filepath} not found, skip ...")

if datasets:
    current_date = datetime.now().strftime('%d-%m-%Y at %H:%M')

    final_ds = xr.concat(datasets, dim="z")
    final_ds.attrs['sigma'] = sigma
    final_ds.attrs['params'] = params
    final_ds.attrs['rolling'] = wx
    final_ds.attrs['Summary'] = f'Created {current_date}'
    final_ds.to_netcdf(os.path.join(combined_outpath, combined_filename))
    print(f"Saved: {combined_outpath + combined_filename}")
else:
    print("No files found.")


depth levels: 100%|██████████| 2/2 [00:00<00:00,  4.63it/s]


Saved: /gxfs_work/geomar/smomw523/eddytools/results/INALT60.L120-KRS0020/smoothed/15/1d/Parcels-mask_depths-2_20120101_20120125_OW0.3_Npix-720-18000_rolling-600.nc


In [37]:
final_ds

<xarray.Dataset>
Dimensions:       (z: 2, time_counter: 25, y: 1483, x: 2404)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2012-01-01 ... 2012-01-25
    nav_lon       (x) float32 0.05 0.06667 0.08333 0.1 ... 40.07 40.08 40.1
    nav_lat       (y) float32 -44.97 -44.95 -44.94 ... -24.95 -24.94 -24.92
    nav_lev       float32 0.4768
Dimensions without coordinates: z, y, x
Data variables:
    eddymask      (z, time_counter, y, x) int8 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    area          (z, time_counter, y, x) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    type          (z, time_counter, y, x) int8 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    scale         (z, time_counter, y, x) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    sigma:    15
    params:   OW0.3_Npix-720-18000
    rolling:  600
    Summary:  Created 02-04-2025 at 20:15